In [1]:
# import standard libraries
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

%matplotlib inline

In [2]:
title_basics = pd.read_table('files/tsv_files/title_basics.tsv', sep = '\t', na_values= ['\\N'])

C:\Users\ROG\anaconda3\envs\ML_envi\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# filter only movie titles
movie_titles = title_basics[title_basics['titleType'].isin(['tvMovie','movie'])]

In [4]:
# remove adult titles
movie_titles = movie_titles[movie_titles['isAdult'] == 0]
# remove end year as this is not needed.
movie_titles = movie_titles.drop(columns=['isAdult','endYear'])

In [5]:
# load ratings data
title_ratings = pd.read_table('files/tsv_files/title_ratings.tsv', na_values= ['\\N'])

In [6]:
# merge with ratings
movie_titles = movie_titles.merge(title_ratings, how = 'left', on = 'tconst')

In [7]:
# get those with at least 5000 votes
movie_titles_short = movie_titles[movie_titles['numVotes'] > 500]
movie_titles_short.shape

(56890, 9)

In [8]:
# drop those without run time minutes
movie_titles_short.dropna(inplace = True)

C:\Users\ROG\anaconda3\envs\ML_envi\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


### Check aggregated data

In [9]:
name_basics = pd.read_table('files/tsv_files/name_basics.tsv', na_values= ['\\N'])
title_crew = pd.read_table('files/tsv_files/title_crew.tsv', na_values= ['\\N'])
title_principals = pd.read_table('files/tsv_files/title_principals.tsv', na_values= ['\\N'])

In [10]:
# select actors|actresses
title_principals = title_principals[title_principals['category'].isin(['actor','actress'])]

#### Directors

In [11]:
movie_directors = title_crew[title_crew['tconst'].isin(movie_titles_short['tconst'])]
movie_directors = movie_directors[movie_directors['directors'].notnull()]

In [12]:
# move into separate rows and join with name
movie_directors['directors_list'] = movie_directors['directors'].apply(lambda x: x.split(','))
movie_directors = movie_directors.explode('directors_list').merge(name_basics[['nconst','primaryName']], how = 'left', left_on = 'directors_list', right_on = 'nconst')

In [13]:
directors = pd.read_csv('files/agg_data/directors.csv')
directors.head()

,Unnamed: 0,primaryName,averageRating,numVotes,tconst,weighted_rating
0,0,A.R. Murugadoss,7.069231,16577.923077,13,6.679422
1,1,Aamir Khan,8.400000,185065.000000,1,7.995856
2,2,Aanand L. Rai,6.820000,24579.000000,5,6.614173
3,3,Aaron Blaise,6.900000,105730.000000,1,6.771573
4,4,Aaron Hann,6.000000,52611.000000,1,6.243639


In [14]:
# 250k average vote and at least 2 movies
directors['well_known'] = ((directors['numVotes'] >= 250000) & (directors['tconst'] > 2)).astype(int)
well_known_directors = directors.loc[directors['well_known'] == 1, 'primaryName']

In [15]:
movie_directors['has_well_known_director'] =  (movie_directors['primaryName'].isin(well_known_directors)).astype(int)

In [16]:
# return to single movie per row
movie_directors.drop(columns=['directors_list'], inplace = True)

# combine name into list
directors = movie_directors.groupby('tconst')['primaryName'].apply(lambda x: list(x))
well_known = movie_directors.groupby('tconst')['has_well_known_director'].sum()

# set as index
movie_directors = movie_directors.drop_duplicates(subset='tconst').set_index('tconst')

# add into column
movie_directors['directors_name'] = directors.apply(lambda x : ','.join(x))
movie_directors['has_well_known_director'] = well_known


movie_directors.reset_index(inplace = True)

In [17]:
# merge movie titles
movie_titles_short = movie_titles_short.merge(movie_directors, how = 'left', on = 'tconst')

In [18]:
movie_titles_short.drop(columns=['primaryName'], inplace = True)
movie_titles_short['has_well_known_director'] = movie_titles_short['has_well_known_director'].fillna(0)

In [19]:
# consdier movies with at least 50k votes
movie_titles_short[movie_titles_short['numVotes'] > 10000].groupby(movie_titles_short['has_well_known_director'] > 0).agg({'tconst':'count','averageRating':'mean'})

,tconst,averageRating
has_well_known_director,,
False,9252,6.622914
True,563,7.208171


#### Actors

In [20]:
# must be in top 3 cast
movie_actors =  title_principals[(title_principals['tconst'].isin(movie_titles_short['tconst'])) & (title_principals['ordering'] <= 3)]
movie_actors.drop(columns=['job'], inplace = True)

C:\Users\ROG\anaconda3\envs\ML_envi\lib\site-packages\pandas\core\frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [21]:
movie_actors = movie_actors.merge(name_basics[['nconst','primaryName']], how = 'left', on = 'nconst')

In [22]:
actors = pd.read_csv('files/agg_data/actors.csv')
actors.head()

,Unnamed: 0,primaryName,averageRating,numVotes,tconst,well_known_actor
0,0,'Ganja' Karuppu,8.10,1899.0,1,0
1,1,'Hurricane Ryu' Hariken,5.95,697.0,2,0
2,2,'Lee' George Quinones,7.10,2588.0,1,0
3,3,'Little Billy' Rhodes,3.90,1749.0,1,0
4,4,'Spring' Mark Adley,6.60,1279.0,1,0


In [23]:
# 250k average vote and at least 2 movies
actors['well_known'] = ((actors['numVotes'] >= 250000) & (actors['tconst'] > 2)).astype(int)
well_known_actors = actors.loc[actors['well_known'] == 1, 'primaryName']

In [24]:
movie_actors['has_well_known_actors'] = (movie_actors['primaryName'].isin(well_known_actors)).astype(int)

# combine into 1 row per movie
primary_actors = movie_actors.groupby('tconst')['primaryName'].apply(lambda x: list(x))
well_known = movie_actors.groupby('tconst')['has_well_known_actors'].sum()

movie_actors = movie_actors.drop_duplicates(subset='tconst').set_index('tconst')

# add into column
movie_actors['primary_actors'] = primary_actors.apply(lambda x: ','.join(x))
movie_actors['has_well_known_actors'] = well_known

movie_actors.reset_index(inplace = True)

In [25]:
movie_actors.drop(columns= ['primaryName','category','characters','ordering'], inplace = True)
movie_titles_short = movie_titles_short.merge(movie_actors, how = 'left', on = 'tconst')

#### Fix columns and data from movie titles short

In [26]:
movie_titles_short.drop(columns=['titleType','nconst_x','nconst_y'], inplace = True)

In [27]:
movie_titles_short['has_well_known_actors'] = movie_titles_short['has_well_known_actors'].fillna(0)

In [28]:
# consdier movies with at least 50k votes
movie_titles_short[movie_titles_short['numVotes'] > 50000].groupby(movie_titles_short['has_well_known_actors'] > 0).agg({'tconst':'count','averageRating':'mean'})

,tconst,averageRating
has_well_known_actors,,
False,3244,6.849630
True,416,7.065385


Having a well known director has larger effect

### Other datasets

Based from the data details, the tmd dataset from kaggle was collected on July 2017. The imdb data that we are working right now is regularly updated. Therefore we might not have data on 2018-2020 movies

In [29]:
# cast, im not sure if this can be used now
# we can get actor and director data from imdb dataset
credits = pd.read_csv('files/tmd/credits.csv')
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [30]:
# link to combine imdb and tmd data
links = pd.read_csv('files/tmd/links.csv')
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [31]:
# convert to int
movie_titles_short['id_int'] = movie_titles['tconst'].str.replace('tt','').astype(int)

#### Combine movie_titles_short with other data.

In [32]:
# around 23.3k missing ids
movie_merge = movie_titles_short.merge(links, how = 'left', left_on= 'id_int', right_on= 'imdbId')

In [33]:
# keywords
keywords = pd.read_csv('files/tmd/keywords.csv')
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [34]:
movie_merge = movie_merge.merge(keywords, how = 'left', left_on='tmdbId', right_on= 'id')

In [35]:
## movies metadaa
metadata = pd.read_csv('files/tmd/movies_metadata.csv')
metadata.head()

C:\Users\ROG\anaconda3\envs\ML_envi\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [36]:
# select only non adult movies
metadata = metadata[metadata['adult'] == 'False']

Can remove
- poster_path: image of poster
- id: already has existing imdb id

In [37]:
metadata.drop(columns= ['poster_path','id','adult'], inplace = True)

In [38]:
movie_merge = movie_merge.merge(metadata, how = 'left', left_on= 'tconst', right_on='imdb_id')

In [39]:
movie_merge.columns

Index(['tconst', 'primaryTitle', 'originalTitle', 'startYear',
       'runtimeMinutes', 'genres_x', 'averageRating', 'numVotes', 'directors',
       'writers', 'has_well_known_director', 'directors_name',
       'has_well_known_actors', 'primary_actors', 'id_int', 'movieId',
       'imdbId', 'tmdbId', 'id', 'keywords', 'belongs_to_collection', 'budget',
       'genres_y', 'homepage', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [40]:
# genres_y is complete on metadata. 
movie_merge = movie_merge[movie_merge['genres_y'].notnull()]

## Clean data

In [41]:
import ast

### Check data from JSON

In [42]:
def check_json(col, sample = 5):
    for json_data in movie_merge.sample(sample)[col]:
        print(json_data)

In [43]:
cols = ['keywords', 'production_countries','production_companies','spoken_languages','genres_y','belongs_to_collection']
for col in cols: 
    print('*' * 100)
    print(col)
    check_json(col)

****************************************************************************************************
keywords
nan
nan
nan
[{'id': 212, 'name': 'london england'}, {'id': 164593, 'name': 'nightclub owner'}]
nan
****************************************************************************************************
production_countries
[{'iso_3166_1': 'US', 'name': 'United States of America'}]
[{'iso_3166_1': 'US', 'name': 'United States of America'}]
[{'iso_3166_1': 'US', 'name': 'United States of America'}]
[{'iso_3166_1': 'US', 'name': 'United States of America'}]
[{'iso_3166_1': 'US', 'name': 'United States of America'}]
****************************************************************************************************
production_companies
[{'name': 'Europa Film', 'id': 1763}]
[]
[{'name': 'DesperaDo', 'id': 7831}]
[]
[]
****************************************************************************************************
spoken_languages
[]
[{'iso_639_1': 'en', 'name': 'English'}]
[{'iso_

In [44]:
# fill keywords with []
movie_merge['keywords'] =  movie_merge['keywords'].fillna('[]')

In [45]:
def clean_belongs_to_collection(entry):
    if str(entry) == 'nan':
        return '[]'
    else: 
        return '[' + str(entry) + ']'

movie_merge['belongs_to_collection'] = movie_merge['belongs_to_collection'].apply(clean_belongs_to_collection)

In [46]:
# get elements from json
def get_list_from_json(string, key = 'name'):
    elements_list = []
    try:
        element_json =  ast.literal_eval(string)
        for dictionary in element_json:
            elements_list.append(dictionary[key])
    except ValueError:
        return 'Error'

    return elements_list

In [47]:
movie_merge['production_companies'] = movie_merge['production_companies'].apply(get_list_from_json)
movie_merge['production_countries'] = movie_merge['production_countries'].apply(get_list_from_json, key = 'iso_3166_1')
movie_merge['keywords'] = movie_merge['keywords'].apply(get_list_from_json)
movie_merge['spoken_languages'] = movie_merge['spoken_languages'].apply(get_list_from_json)
movie_merge['genres_y'] = movie_merge['genres_y'].apply(get_list_from_json)
movie_merge['belongs_to_collection'] =  movie_merge['belongs_to_collection'].apply(get_list_from_json)

### Drop some columns

In [51]:
print(movie_merge.columns)

Index(['tconst', 'primaryTitle', 'originalTitle', 'startYear',
       'runtimeMinutes', 'genres_x', 'averageRating', 'numVotes', 'directors',
       'writers', 'has_well_known_director', 'directors_name',
       'has_well_known_actors', 'primary_actors', 'id_int', 'movieId',
       'imdbId', 'tmdbId', 'id', 'keywords', 'belongs_to_collection', 'budget',
       'genres_y', 'homepage', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'tagline', 'video', 'vote_average', 'vote_count'],
      dtype='object')


In [52]:
movie_merge = movie_merge.drop(columns=['writers','directors','movieId','imdbId','tmdbId','id','imdb_id','original_title','video'])

In [49]:
# drop things that might not be used in the model
movie_merge = movie_merge.drop(columns=['status','title'])

### Feature Engineering

In [56]:
def compute_rating(col, numVotes = 0):
    if numVotes == 0:
        df = movie_merge.groupby(col).agg({'tconst':'count','averageRating':'mean','weighted_rating':'mean'})
        df.columns = ['num_movies','averageRating','averageWeightedRating']
    else: 
        df = movie_merge[movie_merge['numVotes'] >= numVotes].groupby(col).agg({'tconst':'count','averageRating':'mean','weighted_rating':'mean'})
        df.columns = ['num_movies','averageRating','averageWeightedRating']
    return df

In [57]:
# use minimum as 25000
def compute_weighted_rating(row, min = 25000, C = 6.5):
    if (row['numVotes'] == np.nan) | (row['averageRating'] == np.nan): 
        return -1
    wr = ((row['numVotes']/ (row['numVotes'] + min)) * row['averageRating']) + ((min / (min + row['numVotes'])) * C) 
    return wr 

In [58]:
movie_merge['weighted_rating'] =  movie_merge.apply(compute_weighted_rating, axis = 1)

#### Date

In [59]:
movie_merge['release_date'] = pd.to_datetime(movie_merge['release_date'])

In [60]:
# use release date instead
movie_merge.drop(columns=['startYear'], inplace = True)

#### Budget

In [61]:
movie_merge['budget'] = movie_merge['budget'].astype(float)

Adjust for inflation for year 2018

In [62]:
import cpi 
cpi.update()

In [63]:
movie_merge['release_date'].max()

Timestamp('2018-12-31 00:00:00')

In [64]:
def adjust_inflation(row, column, year_now = 2018):
    budget = row[column]
    year = row['release_date'].year

    try: 
        inflated_value = cpi.inflate(budget, year, to = year_now)
        return inflated_value
    except:
        return np.nan

In [65]:
movie_merge['adjusted_budget'] =  movie_merge.apply(adjust_inflation, column = 'budget', axis = 1)

In [66]:
np.corrcoef(movie_merge.loc[movie_merge['adjusted_budget'] > 0,'adjusted_budget'], movie_merge.loc[movie_merge['adjusted_budget'] > 0,'averageRating'])

array([[1.        , 0.10020364],
       [0.10020364, 1.        ]])

#### Revenue

Same calculation with budget

In [67]:
movie_merge['adjusted_revenue'] =  movie_merge.apply(adjust_inflation, column = 'revenue',  axis = 1)

#### RuntimeMinutes

In [68]:
# just need to convert to float
movie_merge['runtimeMinutes'] = movie_merge['runtimeMinutes'].astype(float)

In [69]:
movie_merge.columns

Index(['tconst', 'primaryTitle', 'originalTitle', 'runtimeMinutes', 'genres_x',
       'averageRating', 'numVotes', 'has_well_known_director',
       'directors_name', 'has_well_known_actors', 'primary_actors', 'id_int',
       'keywords', 'belongs_to_collection', 'budget', 'genres_y', 'homepage',
       'original_language', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'tagline', 'vote_average', 'vote_count',
       'weighted_rating', 'adjusted_budget', 'adjusted_revenue'],
      dtype='object')

#### Belongs to collection

Either 1 or 0

In [70]:
movie_merge['belongs_to_collection'] = movie_merge['belongs_to_collection'].apply(lambda x: len(x))

#### Original Language

English or not

In [71]:
movie_merge['original_english'] = (movie_merge['original_language'] == 'en').astype(int)

#### Production companies

In [72]:
movie_merge['has_production_companies'] = movie_merge['production_companies'].apply(lambda x: int(len(x) > 0))

In [73]:
# get only top 3 production companies per movie
movie_merge['production_companies'] = movie_merge['production_companies'].apply(lambda x: x[:3])

In [74]:
prod_companies =  movie_merge[['tconst','primaryTitle','production_companies']]

prod_companies = prod_companies.explode('production_companies')

# https://blog.bizvibe.com/blog/top-movie-production-companies

top_productions = ['Universal Pictures', 'Warner Bros.',
                 'Columbia Pictures','Walt Disney Pictures',
                  'Marvel Productions','Marvel Studios',
                   'Marvel Entertainment', 'Marvel Enterprises',
                   'Paramount Pictures','Twentieth Century Fox Film Corporation',
                   'RatPac-Dune Entertainment','Legendary Pictures','Legendary Entertainment',
                   'Relativity Media']

prod_companies['is_top_production_company'] = prod_companies['production_companies'].isin(top_productions).astype(int)

is_top_production = prod_companies.groupby('tconst')['is_top_production_company'].sum()

prod_companies = prod_companies.drop_duplicates(subset='tconst').set_index('tconst')

prod_companies['is_top_production_company'] = is_top_production


prod_companies.reset_index(inplace = True)

In [75]:
# merge with production company
movie_merge = movie_merge.merge(prod_companies[['tconst','is_top_production_company']], how = 'left', on = 'tconst')

In [76]:
movie_merge['is_top_production_company'] = movie_merge['is_top_production_company'].fillna(0)

#### Production countries

Count number of production countries

In [77]:
def count_production_countries(country_list):
    num_countries = len(country_list)
    if num_countries == 0:
        return '0'
    elif num_countries == 1:
        return '1'
    elif num_countries == 2:
        return '2'
    elif num_countries >= 3:
        return '3+'
    else:
        return -1
    

In [78]:
movie_merge['production_countries'] = movie_merge['production_countries'].apply(lambda x: len(x))

#### Release date

Add release month

In [79]:
movie_merge['release_month'] =  movie_merge['release_date'].dt.month

#### Tagline

Just check if tagline isa available

In [80]:
movie_merge['has_tag_line'] =  movie_merge['tagline'].notnull().astype(int)

#### Spoken languages

Use the same with production countries

In [81]:
movie_merge['spoken_languages'] = movie_merge['spoken_languages'].apply(lambda x: len(x))

#### Homepage

Just check if the homepage is available or not

In [82]:
movie_merge['homepage'] = movie_merge['homepage'].notnull().astype(int)

In [83]:
movie_merge['has_well_known_director'] = (movie_merge['has_well_known_director'] > 0).astype(int)
movie_merge['has_well_known_actors'] = (movie_merge['has_well_known_actors'] > 0).astype(int)
movie_merge['is_top_production_company'] = (movie_merge['is_top_production_company'] > 0).astype(int)

#### Save data

In [84]:
# movie_merge.to_csv('merged_movies.csv', index = False)